In [3]:
import os
import time
import datetime

import pandas as pd
import polars as pl
import numpy as np

# Setup

In [ ]:
DATA_PATH = "../data"
RAW_DATA_DIR = os.path.join(DATA_PATH, "input", "raw")

In [ ]:
df_content = pd.read_csv(os.path.join(RAW_DATA_DIR, "content.csv"))
df_content

In [ ]:
df_topics = pd.read_csv(os.path.join(RAW_DATA_DIR, "topics.csv"))
df_topics

In [ ]:
df_correlations = pd.read_csv(os.path.join(RAW_DATA_DIR, "correlations.csv"))
df_correlations

In [ ]:
df_sample_submission = pd.read_csv(os.path.join(RAW_DATA_DIR, "sample_submission.csv"))
df_sample_submission

# Fold0

In [3]:
df_valid_topic = pd.read_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_topics_flod0_no_source.pqt")
df_valid_topic

,id,title,description,channel,category,level,language,parent,has_content
0,t_0008768bdee6,100 સુધીનો સરવાળો,37 અને 49 જેવી બે-અંકની સંખ્યાઓ ઉમેરતા શીખો.,5223e0,supplemental,4,gu,t_0da7a331d666,True
1,t_0015856edda8,Javascript,None,fc9105,supplemental,5,es,t_121a47996712,False
2,t_00165488ad43,Significance tests (hypothesis testing),Significance tests give us a formal process fo...,2ee29d,aligned,3,en,t_a7a47223471d,False
3,t_001bcbb22694,Lección 1,None,6e90a7,aligned,6,es,t_5d10d6819e04,True
4,t_00250c0c6e1a,Introduction to 3D figures,None,274269,supplemental,4,en,t_f9607325600a,False
...,...,...,...,...,...,...,...,...,...
6709,t_ffdf7dfd64fc,3 Addition,None,8bbad6,supplemental,3,en,t_a34923b85518,False
6710,t_fff05585df72,11: Systems of Equations and Inequalities,None,1fb613,supplemental,4,en,t_5ab3d2eac617,True
6711,t_fff51448598c,Comparative advantage and the gains from trade,Comparative advantage is the ability to produc...,2ee29d,aligned,4,en,t_9d906e167aea,True
6712,t_fff80f4eee89,Polynomial division,"After we have added, subtracted, and multiplie...",6b09a4,supplemental,5,en,t_e93f4fad3893,False


In [4]:
df_valid_label = pd.read_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_correlations_flod0.pqt")
df_valid_label

,topic_id,content_ids
0,t_00068291e9a4,c_639ea2ef9c95
1,t_00068291e9a4,c_89ce9367be10
2,t_00068291e9a4,c_ac1672cdcd2c
3,t_00068291e9a4,c_ebb7fdf10a7e
4,t_0008768bdee6,c_34e1424229b4
...,...,...
55073,t_fff51448598c,c_ab59efb09f73
55074,t_fff51448598c,c_cef3d40736d3
55075,t_fff51448598c,c_d706cb35bd02
55076,t_fff51448598c,c_e346107f23d9


In [5]:
df_valid_label = df_valid_label[df_valid_label["topic_id"].isin(df_valid_topic["id"].unique())]
df_valid_label

,topic_id,content_ids
4,t_0008768bdee6,c_34e1424229b4
5,t_0008768bdee6,c_7d1a964d66d5
6,t_0008768bdee6,c_aab93ee667f4
12,t_001bcbb22694,c_1d9dfc709413
22,t_002dfcaaf1d7,c_7c35d77064e5
...,...,...
55073,t_fff51448598c,c_ab59efb09f73
55074,t_fff51448598c,c_cef3d40736d3
55075,t_fff51448598c,c_d706cb35bd02
55076,t_fff51448598c,c_e346107f23d9


In [6]:
df_valid_label.reset_index(drop=True).to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_correlations_flod0_no_source.pqt")

# Raw

In [4]:
df_label = pd.read_csv("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/raw/correlations.csv")
df_label

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [5]:
df_raw_topic = pd.read_csv("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/raw/topics.csv")
df_raw_topic

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,NaN,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lección 7,NaN,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True


In [6]:
df_raw_topic_without_source = df_raw_topic[df_raw_topic["category"]!="source"]
df_raw_topic_without_source

,id,title,description,channel,category,level,language,parent,has_content
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False
5,t_0008768bdee6,100 સુધીનો સરવાળો,37 અને 49 જેવી બે-અંકની સંખ્યાઓ ઉમેરતા શીખો.,5223e0,supplemental,4,gu,t_0da7a331d666,True
6,t_0008a1bd84ba,12. 20: Bird Reproduction,NaN,ebc86c,supplemental,5,en,t_c44ac9711007,True
8,t_000d1fb3f2f5,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True
10,t_00102869fbcb,Triangles and polygons,Learning outcomes: students must be able to so...,a91e32,aligned,3,en,t_039cecc12bb8,True
...,...,...,...,...,...,...,...,...,...
76964,t_fff80f4eee89,Polynomial division,"After we have added, subtracted, and multiplie...",6b09a4,supplemental,5,en,t_e93f4fad3893,False
76966,t_fff9e5407d13,NA_U06 - El periódico,NaN,71fd51,supplemental,2,es,t_5bd8f6ae9f7d,True
76967,t_fffb0bf2801d,4.3 Graph of functions,NaN,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76969,t_fffe14f1be1e,Lección 7,NaN,6e90a7,aligned,6,es,t_d448c707984d,True


In [7]:
df_label_without_source = df_label[df_label["topic_id"].isin(df_raw_topic_without_source["id"].unique())].reset_index(drop=True)
df_label_without_source

,topic_id,content_ids
0,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
1,t_0008a1bd84ba,c_7ff92a954a3d c_8790b074383e
2,t_000d1fb3f2f5,c_07f1d0eec4b2 c_15a6fb858696 c_175e9db3fc44 c...
3,t_00102869fbcb,c_005e793bd0c5 c_066737fa5146 c_08b2070f92e0 c...
4,t_0012a45fa09c,c_dde078b8ea7a
...,...,...
24998,t_fff1f01cfeb0,c_6cbb967868c2 c_d49c75c4d6f5
24999,t_fff51448598c,c_054e15d2ff3f c_113437812aa0 c_9c817a2bcb3c c...
25000,t_fff7782561f4,c_bca8280a9ad1
25001,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...


In [8]:
df_label_without_source["content_count"] = df_label_without_source["content_ids"].apply(lambda x: len(x.split()))
df_label_without_source

,topic_id,content_ids,content_count
0,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4,3
1,t_0008a1bd84ba,c_7ff92a954a3d c_8790b074383e,2
2,t_000d1fb3f2f5,c_07f1d0eec4b2 c_15a6fb858696 c_175e9db3fc44 c...,26
3,t_00102869fbcb,c_005e793bd0c5 c_066737fa5146 c_08b2070f92e0 c...,87
4,t_0012a45fa09c,c_dde078b8ea7a,1
...,...,...,...
24998,t_fff1f01cfeb0,c_6cbb967868c2 c_d49c75c4d6f5,2
24999,t_fff51448598c,c_054e15d2ff3f c_113437812aa0 c_9c817a2bcb3c c...,8
25000,t_fff7782561f4,c_bca8280a9ad1,1
25001,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...,10


In [18]:
dict(df_label_without_source["content_count"].value_counts())

{1: 7888,
 2: 4265,
 3: 3296,
 4: 2453,
 5: 1601,
 6: 1362,
 7: 970,
 8: 743,
 9: 540,
 10: 485,
 11: 222,
 12: 194,
 13: 143,
 14: 132,
 15: 95,
 16: 68,
 17: 66,
 18: 48,
 19: 46,
 21: 38,
 20: 31,
 26: 27,
 22: 26,
 23: 20,
 29: 19,
 24: 19,
 25: 14,
 30: 12,
 37: 11,
 27: 11,
 36: 11,
 33: 9,
 28: 9,
 31: 8,
 34: 7,
 39: 6,
 40: 5,
 46: 5,
 43: 5,
 49: 5,
 32: 5,
 56: 4,
 45: 4,
 41: 4,
 44: 4,
 53: 4,
 35: 4,
 42: 4,
 48: 3,
 68: 3,
 47: 3,
 74: 2,
 54: 2,
 77: 2,
 57: 2,
 55: 2,
 70: 2,
 38: 2,
 61: 2,
 106: 2,
 91: 1,
 128: 1,
 87: 1,
 105: 1,
 80: 1,
 88: 1,
 69: 1,
 51: 1,
 50: 1,
 170: 1,
 84: 1,
 152: 1,
 64: 1,
 93: 1,
 66: 1,
 192: 1,
 75: 1,
 266: 1,
 71: 1,
 67: 1,
 65: 1,
 150: 1,
 78: 1,
 142: 1,
 123: 1,
 60: 1,
 90: 1,
 58: 1}